In [28]:
#Ayush Joshi and Nikhil Mathur
import heapq
import pygame
import random
import os
import math
import time
from random import *
from random import randint
from random import randrange
from ast import literal_eval as make_tuple
from IPython.core.debugger import set_trace
import numpy as np

In [29]:
# use this instead of randoms
#randoms = list(np.random.randint(3,size=(1000000)))
#print(len(randoms))
#print(randoms.pop())
#print(len(randoms))
def get_random():
    randoms = ()
    if len(randoms) == 0:
        #set_trace()
        randoms = list(np.random.randint(3,size=(100)))
        return randoms.pop()
    else:
        return randoms.pop()

In [30]:
#Convention
# Initialize Grid
# 0 is blocked and 1 is unblocked for maze
# 0 is unvisited and 1 is visited for track_maze
# gn = get neighbour
# nn = new neighbour

In [31]:
# Initialize Screen

blockwidth = 8  # Drawing dimensions of block
GridCols = 101
GridRows = 101


In [32]:
maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
track_maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
start_x = 0
start_y = 0
last_x = 0
last_y = 0
cursor_x = 0
cursor_y = 0
i = 0

In [33]:
tree = {}
# tree["5,5"] = ["5,4","5,6"]
# tree["4,4"] = ["5,5"]
# print(tree)
# tree["5,5"].append("6,5")
# print(tree)

In [34]:
#get the string for the cell location
def get_String(x,y):
    return str(x)+","+str(y)

def get_coord(cell):
    li = cell.split(",")
    return int(li[0]),int(li[1])

#check if the cell is visited or not.
def is_visited(x,y):
    if x is None and y is None:
        return False
    if track_maze[x][y] == 1:
        return True
    return False

def get_key(value):
    #print(value)
    key_value_dict = {}
    for i in tree.items():
        a = i[0]
        b = i[1]
        #print(a,b)
        if a and b:
            for item in b:
                key_value_dict[str(item)] = str(a)
    #print(key_value_dict,key_value_dict[value])
    return key_value_dict[value]

In [35]:
def valid_coordinates(x,y):
    if x is None and y is None:
        return False
    if 0<= x <101 and 0<= y <101:
        return True
    return False
    
#get the neighbours for a cell.
def get_neighbour(x,y,point):
    x= int(x)
    y= int(y)
    
    if point == 0 and valid_coordinates(x-1,y):
        return x-1,y
    if point == 1 and valid_coordinates(x+1,y):
        return x+1,y
    if point == 2 and valid_coordinates(x,y-1):
        return x,y-1
    if point == 3 and valid_coordinates(x,y+1):
        return x,y+1
    return None,None

#check if the cell has neighbours that are not visited.
def validate_neighbour(x,y):
    x=int(x)
    y=int(y)
    tf_value = False    
        
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            tf_value = True
    except:
        pass  
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            tf_value = True
    except:
        pass
        
    return tf_value

def get_validate_neighbour(x,y):
    x = int(x)
    y = int(y)
    
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            return x-1, y
    except:
        pass
    
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            return x+1, y
    except:
        pass
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            return x, y-1
    except:
        pass
    
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            return x, y+1
    except:
        pass
    
    return None, None


In [36]:

#Call to build the tree, backtrack and then call again this recursively.
def build_tree(p_x,p_y):
    nn_x = None
    nn_y = None
    last_p_x = p_x
    last_p_y = p_y
    print("Initially",p_x,p_y)
    #ra = get_random()
    #nn_x,nn_y = get_neighbour(p_x,p_y,ra)
    while validate_neighbour(p_x,p_y) is True:
        print("begining",validate_neighbour(p_x,p_y))
        ra = get_random()
        nn_x,nn_y = get_neighbour(p_x,p_y,ra)
        #nn_x,nn_y  = get_validate_neighbour(p_x,p_y)
        print("neighbour of",p_x,p_y,"is",nn_x,nn_y, "and random", ra)
        if nn_x is None or nn_y is None or is_visited(nn_x,nn_y):
            while validate_neighbour(p_x,p_y):
                #nn_x,nn_y  = get_validate_neighbour(p_x,p_y) #get directly the neighbour
                ran = randint(0,3)
                nn_x,nn_y = get_neighbour(p_x,p_y,ran)
                print("while","neighbour of",p_x,p_y,"is",nn_x,nn_y,"and",is_visited(nn_x,nn_y),"and random is",ran)
                if valid_coordinates(nn_x,nn_y):
                    print("inside",track_maze[int(nn_x)][int(nn_y)])
                    print(nn_x,nn_y)
                    if is_visited(nn_x,nn_y) == False:
                        print("breaking",nn_x,nn_y)
                        break

            print("values","parent-",p_x,p_y,"child-",nn_x,nn_y, "have neighbour", get_validate_neighbour(p_x,p_y))
        if valid_coordinates(nn_x,nn_y) and is_visited(nn_x,nn_y) == False:
            print("I am here adding")
            tree[str(p_x)+","+str(p_y)] = [get_String(nn_x,nn_y)]
            #track_maze[p_x][p_y] = 1 
            track_maze[nn_x][nn_y] = 1
            last_p_x = p_x
            last_p_y = p_y
            rand_block_unblock = randint(0,100)
            if rand_block_unblock < 30:
                maze[p_x][p_y] = 1 # to be changed to 0, using to track
            else:
                maze[p_x][p_y] = 1
            p_x,p_y = nn_x,nn_y
            
    print("build returns",last_p_x, last_p_y )
    return last_p_x, last_p_y

def is_first(x,y):
    if x == start_x and y == start_y:
        return True
    else:
        return False
## Depricated ##
#
# def back_track(last_x,last_y):
#     child_x = last_x
#     child_y = last_y
#     print("Track this",child_x,child_y)
#     nnn_x = None
#     nnn_y = None
#     while is_first(child_x,child_y) == False:
#         parent = get_key(get_String(child_x,child_y))
#         parent_x,parent_y = get_coord(parent)
#         if validate_neighbour(parent_x,parent_y):
#             #use to get the neighbour get_validate_neighbour(x,y)
#             nnn_x,nnn_y = get_validate_neighbour(parent_x,parent_y)
#             if nnn_x is not None and nnn_y is not None:
#                 #print("backtrack this",last_x,last_y)
#                 #print("backtrack found",get_String(nnn_x,nnn_y))
#                 try:
#                     #print("add now",parent_x,parent_y,nnn_x,nnn_y)
#                     tree[str(parent_x)+","+str(parent_y)] = [get_String(int(nnn_x)),int(nnn_y)]
#                     track_maze[nnn_x][nnn_y] = 1                                                                                                         
#                 except:
#                     #print("add now",parent_x,parent_y,nnn_x,nnn_y)
#                     tree[str(parent_x)+","+str(parent_y)] = [get_String(int(nnn_x),int(nnn_y))]
#                     track_maze[nnn_x][nnn_y] = 1
#                 print("backtrack returns",get_String(nnn_x,nnn_y) )
#                 return get_String(nnn_x,nnn_y)
#         child_x,child_y = parent_x,parent_y
#     return None


    

In [37]:
def back_track(last_x,last_y):
    child_x = last_x
    child_y = last_y
    print("Track this",child_x,child_y)
    while is_first(child_x,child_y) == False:
        parent = get_key(get_String(child_x,child_y))
        parent_x,parent_y = get_coord(parent)
        if validate_neighbour(parent_x,parent_y):
            nnn_x,nnn_y = get_validate_neighbour(parent_x,parent_y)
            if nnn_x is not None and nnn_y is not None and is_visited(nnn_x,nnn_y) == False:
                #print("backtrack this",last_x,last_y)
                #print("backtrack found",get_String(nnn_x,nnn_y))
                try:
                    print("add this to tree",get_String(int(nnn_x),int(nnn_y)))
                    print("tree has",tree[str(parent_x)+","+str(parent_y)])
                    newlist = []
                    for item in tree[str(parent_x)+","+str(parent_y)]:
                        newlist.append(item)
                    newlist.append(get_String(int(nnn_x),int(nnn_y)))
                    print("Newlist",newlist)
                    tree[str(parent_x)+","+str(parent_y)] = newlist
                    #tree[str(parent_x)+","+str(parent_y)] = (tree[str(parent_x)+","+str(parent_y)]).append(get_String(int(nnn_x),int(nnn_y)))
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
                    print("\n")
                    #print("add now",parent_x,parent_y,nnn_x,nnn_y)
                except:
                    print("except")
                    #print("add now",parent_x,parent_y,nnn_x,nnn_y)
                    tree[str(parent_x)+","+str(parent_y)] = [get_String(int(nnn_x),int(nnn_y))]
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
            print("backtrack returns",get_String(parent_x,parent_y))
            return get_String(parent_x,parent_y)
        child_x,child_y = parent_x,parent_y
    return None

In [38]:
print(get_String(10,81))

10,81


In [39]:
randint(0,3)

1

In [40]:
#makeGrid()

In [41]:
#back_track(0,49)

In [42]:
is_visited(0,83)

False

In [43]:
# Generate Start and Finish
def generateStartFinish():
	# Generate Start
	x = randint(0,39)
	y = randint(0,39)
	if x>20:
		x = GridCols-x+20
	if y>20:
		y = GridRows-y+20
		
	while maze[x][y]=='a' or maze[x][y]=='b' or maze[x][y]=='0':
		x = randint(0,39)
		y = randint(0,39)
		if x>20:
			x = GridCols-x+20
		if y>20:
			y = GridRows-y+20
		
	start_x = x
	start_y = y
	
	# Generate Finish
	while maze[x][y]=='a' or maze[x][y]=='b' or maze[x][y]=='0' or math.sqrt((x-start_x)**2+(y-start_y)**2)<100:
		x = randint(0,39)
		y = randint(0,39)
		if x>20:
			x = GridCols-x+20
		if y>20:
			y = GridRows-y+20

	goal_x = x
	goal_y = y
	
	return start_x,start_y,goal_x,goal_y


In [44]:
# Draw Grid
def drawGrid(myGridSurface):
	myGridSurface.fill((255,255,255))
	for x in range(len(maze)):
		for y in range(len(maze[x])):
			if maze[x][y] == 0: 
				pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == 1: 
				pygame.draw.rect(myGridSurface, (255,255,255), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == '2': 
				pygame.draw.rect(myGridSurface, (200,200,200), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == 'a': 
				pygame.draw.rect(myGridSurface, (130,170,255), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == 'b': 
				pygame.draw.rect(myGridSurface, (70,90,220), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
	
	myGridSurface = myGridSurface.convert()
	return myGridSurface
	


In [45]:
# Draw Screen
def drawScreen(GridSurface,closed_list,path,pathcost,nodes_expanded,mode,elapsedtime,fn_g,fn_f,fn_h):
	
	GameScreen.blit(GridSurface,(0,0))
	
	#pygame.draw.circle(GameScreen, (255,0,0), int(start_x*blockwidth+blockwidth/2+10,start_y*blockwidth+blockwidth/2+10),blockwidth/2, 0)
	#pygame.draw.circle(GameScreen, (0,0,255), int(goal_x*blockwidth+blockwidth/2+10,goal_y*blockwidth+blockwidth/2+10),blockwidth/2, 0)
	
	pygame.draw.rect(GameScreen, (255,0,0), (cursor_x*blockwidth+9,cursor_y*blockwidth+9,blockwidth+2,blockwidth+2), 2)
	
	# Draw text
	label = myfont.render("G = New Grid | E = New Endpoints | U = Uniform Cost | A = A* Search | W = Weighted A* | V = Visited | S = Save | L = Load", 1, (0,0,0))
	GameScreen.blit(label, (20, blockwidth*GridRows+14))
	
	#label = myfont.render("Cell: ("+str(cursor_x)+","+str(cursor_y)+")", 1, (0,0,0))
	#GameScreen.blit(label, (10+blockwidth*GridCols+30, 30))
	
	#label = myfont.render("Type: "+maze[cursor_x][cursor_y], 1, (0,0,0))
	#GameScreen.blit(label, (10+blockwidth*GridCols+30, 50))
	
	
	label = myfont.render("Start: ("+str(start_x)+","+str(start_y)+")", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 80))
	label = myfont.render("Goal: ("+str(goal_x)+","+str(goal_y)+")", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 100))
	
	if pathcost != 0:
		label = myfont.render("Path Cost:", 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 130))
		label = myfont.render(str(pathcost), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 150))
	
	if pathcost != 0:
		label = myfont.render("Nodes Expanded:", 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 170))
		label = myfont.render(str(nodes_expanded), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 190))
	
	if (cursor_x,cursor_y) in fn_f:
		label = myfont.render("f: "+str(fn_f[(cursor_x,cursor_y)]), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 220))
	
	if (cursor_x,cursor_y) in fn_g:
		label = myfont.render("g: "+str(fn_g[(cursor_x,cursor_y)]), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 240))
	
	if (cursor_x,cursor_y) in fn_h:
		label = myfont.render("h: "+str(fn_h[(cursor_x,cursor_y)]), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 260))
	
	label = myfont.render("Time: ", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 290))
	label = myfont.render(str(elapsedtime*1000)+" ms", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 310))
	
	label = myfont.render("Neighbors:", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 340))
	
	draw_y = 360
	neighbors = getNeighbors(cursor_x,cursor_y)
	
	for neighbor in neighbors:
		label = myfont.render("("+str(neighbor[0])+","+str(neighbor[1])+")", 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, draw_y))
		draw_y += 20
	
	# Draw Final Path and Closed List
	if mode == 1:
		for cell in closed_list:
			pygame.draw.circle(GameScreen, (0,255,0), (cell[0]*blockwidth+blockwidth/2+10,cell[1]*blockwidth+blockwidth/2+10),blockwidth/4, 0)
	for cell in path:
		pygame.draw.circle(GameScreen, (255,0,0), (cell[0]*blockwidth+blockwidth/2+10,cell[1]*blockwidth+blockwidth/2+10),blockwidth/4, 0)
	
	pygame.display.update()


In [46]:
# Get neighbors of a cell
def getNeighbors(x,y):
	myneighbors = [(x-1,y-1),(x,y-1),(x+1,y-1),(x-1,y),(x+1,y),(x-1,y+1),(x,y+1),(x+1,y+1)]
	myneighbors[:] = [neighbor for neighbor in myneighbors if neighbor[0]>=0 and neighbor[1]>=0 and neighbor[0]<GridCols and neighbor[1]<GridRows and maze[neighbor[0]][neighbor[1]] != '0']
	
	return myneighbors

def cost(currx, curry, nextx, nexty):
	cost = 0.0
	
	if (maze[currx][curry] == 'a' or maze[currx][curry] == '1') and (maze[nextx][nexty] == 'a' or maze[nextx][nexty] == '1'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = math.sqrt(2)
		else:
			#traversing unblocked diagonally
			cost = 1
	if (maze[currx][curry] == 'b' or maze[currx][curry] == '2') and (maze[nextx][nexty] == '2' or maze[nextx][nexty] == 'b'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = math.sqrt(8)
		else:
			#traversing unblocked diagonally
			cost = 2
	if (maze[currx][curry] == 'a' or maze[currx][curry] == '1') and (maze[nextx][nexty] == '2' or maze[nextx][nexty] == 'b'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = (math.sqrt(2) + math.sqrt(8)) / 2
		else:
			#traversing unblocked diagonally
			cost = 1.5
	if (maze[currx][curry] == '2' or maze[currx][curry] == 'b') and (maze[nextx][nexty] == '1' or maze[nextx][nexty] == 'a'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = (math.sqrt(2) + math.sqrt(8)) / 2
		else:
			#traversing unblocked diagonally
			cost = 1.5
	if (maze[currx][curry] == 'a' or maze[currx][curry] == 'b') and (maze[nextx][nexty] == 'a' or maze[nextx][nexty] == 'b') and (((currx == nextx + 1 or currx == nexty - 1) and curry == nexty) or ((curry == nexty + 1 or curry == nexty - 1) and currx == nextx)):
		cost = cost / 4
		return cost
	if cost == 0.0:
		print("Could not find cost value. Current: " + str(currx) + ", " + str(curry) + " of type: " + maze[currx][curry] + "/Next: " + str(nextx) + ", " + str(nexty) + " of type: " + maze[nextx][nexty])


	return cost


In [47]:
# A* Stuff

class Coordinate:
	def __init__(self, x, y, parent):
		self.x = x
		self.y = y
		self.parent = parent
		
	def get_x(self):
		return self.x
		
	def get_y(self):
		return self.y
	
	def get_parent(self):
		return self.parent

class PriorityQueue:
	def __init__(self):
		self.elements = []

	def empty(self):
		return len(self.elements) == 0

	def put(self, item, priority):
		heapq.heappush(self.elements, (priority, item))

	def get(self):
		return heapq.heappop(self.elements)[1]

def heuristic(self,startx,starty,goalx,goaly,choice):
	start = (startx, starty)
	goal = (goalx, goaly)
	choice = int(choice)

	if choice == 1: #manhattan
			heuristic = abs(int(startx) - int(goalx)) + abs(int(starty) - int(goaly))
			heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
			return heuristic
	if choice == 2: #euclidean
			heuristic = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
			heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
			return heuristic
	if choice == 3: #octile
			dx = abs(int(startx) - int(goalx))
			dy = abs(int(starty) - int(goaly))
			heuristic = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
			heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
			return heuristic
	if choice == 4: #Chebyshev
			heuristic = max(abs(startx - goalx), abs(starty - goaly))
			heuristic *= (1.0 + (0.25/160))
			return heuristic
	if choice == 5: #5th heuristic
			heuristic = math.sqrt(2)*min(abs(startx - goalx), abs(starty - goaly)) + max(abs(startx - goalx), abs(starty - goaly)) - min(abs(startx - goalx), abs(starty - goaly))
			heuristic *= (1.0 + (0.25/160))
			return heuristic
	if choice == 6: #Best - minimum of all
			dx = abs(int(startx) - int(goalx))
			dy = abs(int(starty) - int(goaly))
			h1 = dx + dy 
			h2 = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
			h3 = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
			h4 = max(dx, dy)
			h5 = math.sqrt(2)*min(dx, dy) + max(dx, dy) - min(dx, dy)
			h6 = min(h1, h2, h3, h4, h5)
			h6 *= (1.0 + (0.25/160))
			return h6
	return 0


In [48]:
class AStarSearch(object):
	def Search(self, startx, starty, goalx, goaly, choice):
		fringe = PriorityQueue()
		start = Coordinate(startx, starty, None)
		goal = (goalx, goaly)
		fringe.put(start, 0)

		closed_list = {}
		cost_added = {}
		final_path = []
		heuristic_list = {}
		priority_list = {}
		closed_list[(start.get_x(),start.get_y())] = None
		cost_added[(start.get_x(),start.get_y())] = 0

		# f = priority_list[]
		# g = cost_added[]
		# h = heuristic
		
		while not fringe.empty():
			current = fringe.get()
			#print "got current from fringe with x %d and y %d" % current[0], current[1]
			if (current.get_x(),current.get_y()) == goal:
				print("Made it to goal at " + str(goal[0]) + "," + str(goal[1]))
				
				# Make a straight path from goal to start
				PathNode = current
				
				while PathNode != None:
					final_path.append((PathNode.get_x(),PathNode.get_y()))
					#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
					PathNode = PathNode.get_parent()
			
				break

			for next in getNeighbors(current.get_x(), current.get_y()):
				#print "current x %d current y %d" % current[0], current[1]
				new_cost = cost_added[(current.get_x(),current.get_y())] + cost(current.get_x(), current.get_y(), next[0], next[1])
				
				if next not in cost_added or new_cost < cost_added[next]:
				#if next not in closed_list or new_cost < cost_added[next]:
					cost_added[next] = new_cost		# g
					myheuristic = self.heuristic(next[0], next[1], goal_x, goal_y, choice)
					priority = new_cost + myheuristic
					heuristic_list[next] = myheuristic
					priority_list[next] = priority
					fringe.put(Coordinate(next[0],next[1],current), priority)
					closed_list[next] = current
		
		return closed_list, cost_added, final_path, cost_added[(goalx,goaly)], priority_list, heuristic_list

	def heuristic(self,startx,starty,goalx,goaly,choice):
		start = (startx, starty)
		goal = (goalx, goaly)
		choice = int(choice)

		if choice == 1: #manhattan
				heuristic = abs(int(startx) - int(goalx)) + abs(int(starty) - int(goaly))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 2: #euclidean
				heuristic = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 3: #octile
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				heuristic = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 4: #Chebyshev
				heuristic = max(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 5: #5th heuristic
				heuristic = math.sqrt(2)*min(abs(startx - goalx), abs(starty - goaly)) + max(abs(startx - goalx), abs(starty - goaly)) - min(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 6: #Best - minimum of all
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				h1 = dx + dy 
				h2 = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				h3 = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				h4 = max(dx, dy)
				h5 = math.sqrt(2)*min(dx, dy) + max(dx, dy) - min(dx, dy)
				h6 = min(h1, h2, h3, h4, h5)
				h6 *= (1.0 + (0.25/160))
				return h6
		return 0
class AStarSearch(object):
	def Search(self, startx, starty, goalx, goaly, choice):
		fringe = PriorityQueue()
		start = Coordinate(startx, starty, None)
		goal = (goalx, goaly)
		fringe.put(start, 0)

		closed_list = {}
		cost_added = {}
		final_path = []
		heuristic_list = {}
		priority_list = {}
		closed_list[(start.get_x(),start.get_y())] = None
		cost_added[(start.get_x(),start.get_y())] = 0

		# f = priority_list[]
		# g = cost_added[]
		# h = heuristic
		
		while not fringe.empty():
			current = fringe.get()
			#print "got current from fringe with x %d and y %d" % current[0], current[1]
			if (current.get_x(),current.get_y()) == goal:
				print("Made it to goal at " + str(goal[0]) + "," + str(goal[1]))
				
				# Make a straight path from goal to start
				PathNode = current
				
				while PathNode != None:
					final_path.append((PathNode.get_x(),PathNode.get_y()))
					#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
					PathNode = PathNode.get_parent()
			
				break

			for next in getNeighbors(current.get_x(), current.get_y()):
				#print "current x %d current y %d" % current[0], current[1]
				new_cost = cost_added[(current.get_x(),current.get_y())] + cost(current.get_x(), current.get_y(), next[0], next[1])
				
				if next not in cost_added or new_cost < cost_added[next]:
				#if next not in closed_list or new_cost < cost_added[next]:
					cost_added[next] = new_cost		# g
					myheuristic = self.heuristic(next[0], next[1], goal_x, goal_y, choice)
					priority = new_cost + myheuristic
					heuristic_list[next] = myheuristic
					priority_list[next] = priority
					fringe.put(Coordinate(next[0],next[1],current), priority)
					closed_list[next] = current
		
		return closed_list, cost_added, final_path, cost_added[(goalx,goaly)], priority_list, heuristic_list

	def heuristic(self,startx,starty,goalx,goaly,choice):
		start = (startx, starty)
		goal = (goalx, goaly)
		choice = int(choice)

		if choice == 1: #manhattan
				heuristic = abs(int(startx) - int(goalx)) + abs(int(starty) - int(goaly))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 2: #euclidean
				heuristic = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 3: #octile
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				heuristic = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 4: #Chebyshev
				heuristic = max(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 5: #5th heuristic
				heuristic = math.sqrt(2)*min(abs(startx - goalx), abs(starty - goaly)) + max(abs(startx - goalx), abs(starty - goaly)) - min(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 6: #Best - minimum of all
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				h1 = dx + dy 
				h2 = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				h3 = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				h4 = max(dx, dy)
				h5 = math.sqrt(2)*min(dx, dy) + max(dx, dy) - min(dx, dy)
				h6 = min(h1, h2, h3, h4, h5)
				h6 *= (1.0 + (0.25/160))
				return h6
		return 0
				
class UniformCostSearch(AStarSearch):
	def heuristic(self,startx, starty, goalx, goaly, choice):
		return 1

class WeightedAStarSearch(AStarSearch):
	def __init__(self,weight):
		self.weight = weight
	def heuristic(self,startx, starty, goalx, goaly, choice):
		return super(WeightedAStarSearch, self).heuristic(startx,starty,goalx,goaly,choice) * float(self.weight)

#Phase 2 Begin
class SequentialAStarSearch(AStarSearch):
	def Search(self,startx, starty, goalx, goaly, notused):
		fringe = [PriorityQueue() for x in range(5)]
		start = Coordinate(startx, starty, None)
		goal = (goalx, goaly)

		closed_list = [dict() for y in range (0,5)]
		cost_added = [dict() for y in range (0,5)]
		heuristic_list = [dict() for y in range(0,5)]
		final_path = []
		priority_list = [dict() for y in range(0,5)]
		w2 = 1 #weight
		path_cost = 0
		'''closed_list = {}
		cost_added = {}
		final_path = []
		heuristic_list = {}
		priority_list = {}
		closed_list[(start.get_x(),start.get_y())] = None
		cost_added[(start.get_x(),start.get_y())] = 0
		'''
		for i in range(0, 5):
			closed_list[i] = {}
			cost_added[i] = {}
			heuristic_list[i] = {}
			for next in getNeighbors(start.get_x(), start.get_y()):
				#print "current x %d current y %d" % current[0], current[1]
				new_cost = cost(start.get_x(), start.get_y(), next[0], next[1])
				if next not in cost_added[i] or new_cost < cost_added[i][next]:
					#if next not in closed_list or new_cost < cost_added[next]:
					cost_added[i][next] = new_cost		# g
					myheuristic = self.heuristic(next[0], next[1], goal_x, goal_y, i+1)
					priority = new_cost + myheuristic
					heuristic_list[i][next] = myheuristic
					priority_list[i][next] = priority
					fringe[i].put(Coordinate(next[0],next[1],start), priority)
					closed_list[i][next] = start
			closed_list[i][(start.get_x(), start.get_y())] = None
			cost_added[i][(start.get_x(), start.get_y())] = 0
			cost_added[i][(goalx, goaly)] = float("inf")
		# f = priority_list[]
		# g = cost_added[]
		# h = heuristic
		while not fringe[0].empty():
			anchor = fringe[0].getFull()
			for i in range(1, 5):
				#anchor is a tuple (item, priority)
				temp = fringe[i].getFull() #temp is a tuple (item, priority)
				
				'''if temp != -1:
					print "x,y of anchor is: ("+str(anchor[0].get_x())+","+str(anchor[0].get_y())+")"
					print "x,y of temp is: (" + str(temp[0].get_x()) + ","+str(temp[0].get_y())+")"
					print "priority/key of anchor is: " + str(anchor[1])
					print "priority/key of temp is: " + str(temp[1])'''

				#[0] = item
				#[1] = priority

				if temp!= -1 and temp[1] <= w2*anchor[1]: #main condition
					if temp[0].get_x() == goalx and temp[0].get_y() == goaly:
						# Found goal, return path
						print("Made it to goal at " + str(goal[0]) + "," + str(goal[1]))
						path_cost = 0
						# Make a straight path from goal to start
						PathNode = temp[0]
						old_x = PathNode.get_x()
						old_y = PathNode.get_y()
						final_path.append((old_x,old_y))
						PathNode = PathNode.get_parent()

						while PathNode != None:
							path_cost += cost(PathNode.get_x(),PathNode.get_y(),old_x,old_y)
							final_path.append((PathNode.get_x(),PathNode.get_y()))
							#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
							old_x = PathNode.get_x()
							old_y = PathNode.get_y()
							PathNode = PathNode.get_parent()
						
						#break
						return closed_list, cost_added, final_path, path_cost, priority_list, heuristic_list
					else:
						#print "adding to fringe[" + str(i)+"]"
						for next in getNeighbors(temp[0].get_x(), temp[0].get_y()):
							#print "current x %d current y %d" % current[0], current[1]
							new_cost = cost_added[i][(temp[0].get_x(),temp[0].get_y())] + cost(temp[0].get_x(), temp[0].get_y(), next[0], next[1])

							if next not in cost_added[i] or new_cost < cost_added[i][next]:
								#print "Added neighbor"
								#if next not in closed_list or new_cost < cost_added[next]:
								cost_added[i][next] = new_cost		# g
								myheuristic = self.heuristic(next[0], next[1], goalx, goaly, i+1)
								priority = new_cost + myheuristic
								heuristic_list[i][next] = myheuristic
								priority_list[i][next] = priority
								fringe[i].put(Coordinate(next[0],next[1],temp[0]), priority)
								closed_list[i][next] = temp[0]

				else:
					#print "------------- Reached the else condition, using fringe[0] ---------------"
					if anchor[0].get_x() == goalx and anchor[0].get_y() == goaly:
						# Found goal, return path
						print("Made it to goal at" + str(goal[0]) + "," + str(goal[1]))
						path_cost = 0
						# Make a straight path from goal to start
						PathNode = anchor[0]
						old_x = PathNode.get_x()
						old_y = PathNode.get_y()
						final_path.append((old_x,old_y))
						PathNode = PathNode.get_parent()

						while PathNode != None:
							path_cost += cost(PathNode.get_x(),PathNode.get_y(),old_x,old_y)
							final_path.append((PathNode.get_x(),PathNode.get_y()))
							#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
							old_x = PathNode.get_x()
							old_y = PathNode.get_y()
							PathNode = PathNode.get_parent()
						
						#break
						return closed_list, cost_added, final_path, path_cost, priority_list, heuristic_list
					else:
						#print "-------------- adding to fringe[0] ---------------"
						for next in getNeighbors(anchor[0].get_x(), anchor[0].get_y()):
							#print "current x %d current y %d" % current[0], current[1]
							new_cost=cost_added[0][(anchor[0].get_x(), anchor[0].get_y())]+cost(anchor[0].get_x(),anchor[0].get_y(),next[0],next[1])

							if next not in cost_added[0] or new_cost < cost_added[0][next]:
								#print "Added neighbor"
							#if next not in closed_list or new_cost < cost_added[next]:
								cost_added[0][next] = new_cost		# g
								myheuristic = self.heuristic(next[0], next[1], goalx, goaly, 1)
								priority = new_cost + myheuristic
								heuristic_list[0][next] = myheuristic
								priority_list[0][next] = priority
								fringe[0].put(Coordinate(next[0],next[1],anchor[0]), priority)
								closed_list[0][next] = anchor[0]


		return closed_list, cost_added, final_path, path_cost, priority_list, heuristic_list

	def heuristic(self,startx,starty,goalx,goaly,choice):
		super(SequentialAStarSearch, self).heuristic(startx,starty,goalx,goaly,choice)



In [49]:
# Make the grid
#areacoordinates = makeGrid()

In [50]:
i

0

In [51]:
## For visual
pygame.init()
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())
myfont = pygame.font.SysFont("monospace", 15)
start_x,start_y,goal_x,goal_y = generateStartFinish()

In [52]:

# Make Random Grid Visual
def makeGrid_visual():
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        start_x = p_x
        start_y = p_y
        ra = get_random()
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    maze[p_x][p_y] = 1
    track_maze[p_x][p_y] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    # start building the tree. when deadend is reach call the bcaktrack and call again the build tree.
    #while(get_backtrack_cell is not None)
    #call build_tree for the backtrack_cell.
    
    
    while True:
        
        #time.sleep(5)
        
        #pygame.init()
        GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
        GridSurface = pygame.Surface(GameScreen.get_size())
        GridSurface = drawGrid(GridSurface)
        
        final_path = []
        closed_list = []
        cell_costs = []
        priority_list = []
        heuristic_list = []
        path_cost = 0
        elapsed_time = 0
        drawmode = 0
        nodes_expanded = 0
        #print(GridSurface)
        drawScreen(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)
        #time.sleep(2)
        #pygame.quit()
        #i = i +1
        #print("Tree",tree)
        #print("\n")
        new_x,new_y = build_tree(p_x,p_y)
        print("back to main",new_x,new_y)
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == start_x and p_y == start_y:
                    break 
            if new_parent is None:
                return None
    return None


In [53]:

# Make Random Grid Visual
def makeGrid():
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        start_x = p_x
        start_y = p_y
        ra = get_random()
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    maze[p_x][p_y] = 1
    track_maze[p_x][p_y] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    # start building the tree. when deadend is reach call the bcaktrack and call again the build tree.
    #while(get_backtrack_cell is not None)
    #call build_tree for the backtrack_cell.
    
    while True:
        i = i +1
        print("Tree",tree)
        print("\n")
        new_x,new_y = build_tree(p_x,p_y)
        print("back to main",new_x,new_y)
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == start_x and p_y == start_y:
                    break 
            if new_parent is None:
                return None
    return None


In [54]:
makeGrid_visual()

first
Initially 62 68
begining True
neighbour of 62 68 is 63 68 and random 1
while neighbour of 62 68 is 61 68 and False and random is 0
inside 0
61 68
breaking 61 68
values parent- 62 68 child- 61 68 have neighbour (61, 68)
I am here adding
begining True
neighbour of 61 68 is 62 68 and random 1
I am here adding
begining True
neighbour of 62 68 is 61 68 and random 0
while neighbour of 62 68 is 62 69 and False and random is 3
inside 0
62 69
breaking 62 69
values parent- 62 68 child- 62 69 have neighbour (62, 67)
I am here adding
begining True
neighbour of 62 69 is 63 69 and random 1
I am here adding
begining True
neighbour of 63 69 is 64 69 and random 1
I am here adding
begining True
neighbour of 64 69 is 63 69 and random 0
while neighbour of 64 69 is 63 69 and True and random is 0
inside 1
63 69
while neighbour of 64 69 is 64 68 and False and random is 2
inside 0
64 68
breaking 64 68
values parent- 64 69 child- 64 68 have neighbour (65, 69)
I am here adding
begining True
neighbour of 6

while neighbour of 75 29 is 76 29 and True and random is 1
inside 1
76 29
while neighbour of 75 29 is 76 29 and True and random is 1
inside 1
76 29
while neighbour of 75 29 is 75 30 and True and random is 3
inside 1
75 30
while neighbour of 75 29 is 74 29 and False and random is 0
inside 0
74 29
breaking 74 29
values parent- 75 29 child- 74 29 have neighbour (74, 29)
I am here adding
begining True
neighbour of 74 29 is 74 28 and random 2
I am here adding
begining True
neighbour of 74 28 is 74 27 and random 2
I am here adding
begining True
neighbour of 74 27 is 75 27 and random 1
I am here adding
begining True
neighbour of 75 27 is 76 27 and random 1
I am here adding
begining True
neighbour of 76 27 is 75 27 and random 0
while neighbour of 76 27 is 76 28 and True and random is 3
inside 1
76 28
while neighbour of 76 27 is 76 26 and False and random is 2
inside 0
76 26
breaking 76 26
values parent- 76 27 child- 76 26 have neighbour (77, 27)
I am here adding
begining True
neighbour of 76 2

neighbour of 93 7 is 94 7 and random 1
I am here adding
begining True
neighbour of 94 7 is 95 7 and random 1
I am here adding
begining True
neighbour of 95 7 is 96 7 and random 1
I am here adding
begining True
neighbour of 96 7 is 96 6 and random 2
I am here adding
begining True
neighbour of 96 6 is 96 5 and random 2
I am here adding
begining True
neighbour of 96 5 is 97 5 and random 1
I am here adding
begining True
neighbour of 97 5 is 96 5 and random 0
while neighbour of 97 5 is 97 6 and False and random is 3
inside 0
97 6
breaking 97 6
values parent- 97 5 child- 97 6 have neighbour (98, 5)
I am here adding
begining True
neighbour of 97 6 is 97 5 and random 2
while neighbour of 97 6 is 96 6 and True and random is 0
inside 1
96 6
while neighbour of 97 6 is 97 5 and True and random is 2
inside 1
97 5
while neighbour of 97 6 is 98 6 and False and random is 1
inside 0
98 6
breaking 98 6
values parent- 97 6 child- 98 6 have neighbour (98, 6)
I am here adding
begining True
neighbour of 98 

values parent- 95 21 child- 95 22 have neighbour (94, 21)
I am here adding
begining True
neighbour of 95 22 is 96 22 and random 1
I am here adding
begining True
neighbour of 96 22 is 96 21 and random 2
I am here adding
begining True
neighbour of 96 21 is 97 21 and random 1
I am here adding
begining True
neighbour of 97 21 is 97 20 and random 2
I am here adding
begining True
neighbour of 97 20 is 97 19 and random 2
I am here adding
begining True
neighbour of 97 19 is 97 18 and random 2
I am here adding
begining True
neighbour of 97 18 is 97 17 and random 2
I am here adding
begining True
neighbour of 97 17 is 98 17 and random 1
I am here adding
begining True
neighbour of 98 17 is 99 17 and random 1
I am here adding
begining True
neighbour of 99 17 is 98 17 and random 0
while neighbour of 99 17 is 98 17 and True and random is 0
inside 1
98 17
while neighbour of 99 17 is 98 17 and True and random is 0
inside 1
98 17
while neighbour of 99 17 is 99 18 and False and random is 3
inside 0
99 18

neighbour of 86 21 is 85 21 and random 0
I am here adding
begining True
neighbour of 85 21 is 85 20 and random 2
I am here adding
begining True
neighbour of 85 20 is 84 20 and random 0
I am here adding
begining True
neighbour of 84 20 is 84 19 and random 2
while neighbour of 84 20 is 84 19 and True and random is 2
inside 1
84 19
while neighbour of 84 20 is 84 21 and False and random is 3
inside 0
84 21
breaking 84 21
values parent- 84 20 child- 84 21 have neighbour (84, 21)
I am here adding
begining True
neighbour of 84 21 is 83 21 and random 0
I am here adding
begining True
neighbour of 83 21 is 84 21 and random 1
while neighbour of 83 21 is 83 20 and True and random is 2
inside 1
83 20
while neighbour of 83 21 is 82 21 and False and random is 0
inside 0
82 21
breaking 82 21
values parent- 83 21 child- 82 21 have neighbour (82, 21)
I am here adding
begining True
neighbour of 82 21 is 83 21 and random 1
while neighbour of 82 21 is 82 22 and False and random is 3
inside 0
82 22
breaking

...

while neighbour of 60 92 is 59 92 and False and random is 0
inside 0
59 92
breaking 59 92
values parent- 60 92 child- 59 92 have neighbour (59, 92)
I am here adding
begining True
neighbour of 59 92 is 60 92 and random 1
while neighbour of 59 92 is 58 92 and False and random is 0
inside 0
58 92
breaking 58 92
values parent- 59 92 child- 58 92 have neighbour (58, 92)
I am here adding
begining True
neighbour of 58 92 is 59 92 and random 1
while neighbour of 58 92 is 58 93 and False and random is 3
inside 0
58 93
breaking 58 93
values parent- 58 92 child- 58 93 have neighbour (57, 92)
I am here adding
begining True
neighbour of 58 93 is 58 92 and random 2
while neighbour of 58 93 is 59 93 and True and random is 1
inside 1
59 93
while neighbour of 58 93 is 58 92 and True and random is 2
inside 1
58 92
while neighbour of 58 93 is 58 94 and False and random is 3
inside 0
58 94
breaking 58 94
values parent- 58 93 child- 58 94 have neighbour (57, 93)
I am here adding
begining True
neighbour of 

begining True
neighbour of 90 94 is 89 94 and random 0
while neighbour of 90 94 is 90 93 and False and random is 2
inside 0
90 93
breaking 90 93
values parent- 90 94 child- 90 93 have neighbour (91, 94)
I am here adding
begining True
neighbour of 90 93 is 89 93 and random 0
while neighbour of 90 93 is 91 93 and False and random is 1
inside 0
91 93
breaking 91 93
values parent- 90 93 child- 91 93 have neighbour (91, 93)
I am here adding
begining True
neighbour of 91 93 is 90 93 and random 0
while neighbour of 91 93 is 91 94 and False and random is 3
inside 0
91 94
breaking 91 94
values parent- 91 93 child- 91 94 have neighbour (92, 93)
I am here adding
begining True
neighbour of 91 94 is 91 93 and random 2
while neighbour of 91 94 is 92 94 and False and random is 1
inside 0
92 94
breaking 92 94
values parent- 91 94 child- 92 94 have neighbour (92, 94)
I am here adding
begining True
neighbour of 92 94 is 91 94 and random 0
while neighbour of 92 94 is 92 93 and False and random is 2
insid

while neighbour of 96 85 is 96 86 and True and random is 3
inside 1
96 86
while neighbour of 96 85 is 96 86 and True and random is 3
inside 1
96 86
while neighbour of 96 85 is 96 86 and True and random is 3
inside 1
96 86
while neighbour of 96 85 is 97 85 and True and random is 1
inside 1
97 85
while neighbour of 96 85 is 96 84 and True and random is 2
inside 1
96 84
while neighbour of 96 85 is 96 86 and True and random is 3
inside 1
96 86
while neighbour of 96 85 is 97 85 and True and random is 1
inside 1
97 85
while neighbour of 96 85 is 96 86 and True and random is 3
inside 1
96 86
while neighbour of 96 85 is 95 85 and False and random is 0
inside 0
95 85
breaking 95 85
values parent- 96 85 child- 95 85 have neighbour (95, 85)
I am here adding
begining True
neighbour of 95 85 is 96 85 and random 1
while neighbour of 95 85 is 96 85 and True and random is 1
inside 1
96 85
while neighbour of 95 85 is 94 85 and False and random is 0
inside 0
94 85
breaking 94 85
values parent- 95 85 chi

neighbour of 93 96 is 94 96 and random 1
while neighbour of 93 96 is 94 96 and True and random is 1
inside 1
94 96
while neighbour of 93 96 is 93 97 and False and random is 3
inside 0
93 97
breaking 93 97
values parent- 93 96 child- 93 97 have neighbour (92, 96)
I am here adding
begining True
neighbour of 93 97 is 94 97 and random 1
I am here adding
begining True
neighbour of 94 97 is 93 97 and random 0
while neighbour of 94 97 is 94 98 and False and random is 3
inside 0
94 98
breaking 94 98
values parent- 94 97 child- 94 98 have neighbour (95, 97)
I am here adding
begining True
neighbour of 94 98 is 94 97 and random 2
while neighbour of 94 98 is 95 98 and False and random is 1
inside 0
95 98
breaking 95 98
values parent- 94 98 child- 95 98 have neighbour (93, 98)
I am here adding
begining True
neighbour of 95 98 is 95 97 and random 2
I am here adding
begining True
neighbour of 95 97 is 95 96 and random 2
I am here adding
begining True
neighbour of 95 96 is 96 96 and random 1
I am here

while neighbour of 88 98 is 89 98 and True and random is 1
inside 1
89 98
while neighbour of 88 98 is 88 97 and True and random is 2
inside 1
88 97
while neighbour of 88 98 is 88 97 and True and random is 2
inside 1
88 97
while neighbour of 88 98 is 87 98 and False and random is 0
inside 0
87 98
breaking 87 98
values parent- 88 98 child- 87 98 have neighbour (87, 98)
I am here adding
begining True
neighbour of 87 98 is 87 97 and random 2
while neighbour of 87 98 is 86 98 and False and random is 0
inside 0
86 98
breaking 86 98
values parent- 87 98 child- 86 98 have neighbour (86, 98)
I am here adding
begining True
neighbour of 86 98 is 85 98 and random 0
I am here adding
begining True
neighbour of 85 98 is 86 98 and random 1
while neighbour of 85 98 is 85 97 and False and random is 2
inside 0
85 97
breaking 85 97
values parent- 85 98 child- 85 97 have neighbour (84, 98)
I am here adding
begining True
neighbour of 85 97 is 84 97 and random 0
I am here adding
begining True
neighbour of 84

In [55]:
#makeGrid()

In [56]:
# Main loop
running = True
pygame.init()
myfont = pygame.font.SysFont("monospace", 15)
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())

time.sleep(2)
GridSurface = drawGrid(GridSurface)
start_x,start_y,goal_x,goal_y = generateStartFinish()
final_path = []
closed_list = []
cell_costs = []
priority_list = []
heuristic_list = []
path_cost = 0
elapsed_time = 0
drawmode = 0
nodes_expanded = 0

MySearch = AStarSearch() # Initialize Object

while(running):
	# Get Input
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			running = False
		elif event.type == pygame.KEYDOWN:
			if event.key == pygame.K_ESCAPE:
				running = False
			elif event.key == pygame.K_g:
				maze = [['1' for y in range(GridRows)] for x in range(GridCols)]
				areacoordinates = makeGrid()
				GridSurface = drawGrid(GridSurface)
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new map")
			elif event.key == pygame.K_e:
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new start and finish points")
			elif event.key == pygame.K_s:
				# Save map: get filename
				filename = raw_input("Save map to: ")
				with open(os.path.join("./gen",filename),"w") as mapfile:
					mapfile.write(str((start_x,start_y)))		# Write start
					mapfile.write("\n")
					mapfile.write(str((goal_x,goal_y)))			# Write goal
					mapfile.write("\n")
					
					for area in areacoordinates:				# Write hard to traverse area centers
						mapfile.write(str((area[0],area[1])))		
						mapfile.write("\n")
					
					for y in range(len(maze[x])):				# Write each cell
						for x in range(len(maze)):					
							mapfile.write(str(maze[x][y]))		
						mapfile.write("\n")
					
					mapfile.close()
				print("Map saved!")
			elif event.key == pygame.K_l:
				# Load map: get filename
				filename = raw_input("Load map from: ")
				with open(os.path.join("./gen",filename),"r") as mapfile: #changed to allow using /maps folder
					new_start = make_tuple(mapfile.readline())
					start_x = new_start[0]
					start_y = new_start[1]
					new_goal = make_tuple(mapfile.readline())
					goal_x = new_goal[0]
					goal_y = new_goal[1]
					
					areacoordinates = []
					
					for i in range(8):
						new_area = make_tuple(mapfile.readline())
						areacoordinates.append((new_area[0],new_area[1]))
					
					for y in range(len(maze[x])):				# Read each cell
						for x in range(len(maze)):					
							maze[x][y] = mapfile.read(1)
						mapfile.read(1)
					
					mapfile.close()
				final_path = []
				closed_list = []
				cell_costs = []
				priority_list = []
				heuristic_list = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				GridSurface = drawGrid(GridSurface)
				print("Map loaded!")
			elif event.key == pygame.K_UP:
				if cursor_y-1 >= 0:
					cursor_y -= 1
			elif event.key == pygame.K_LEFT:
				if cursor_x-1 >= 0:
					cursor_x -= 1
			elif event.key == pygame.K_RIGHT:
				if cursor_x+1 < GridCols:
					cursor_x += 1
			elif event.key == pygame.K_DOWN:
				if cursor_y+1 < GridRows:
					cursor_y += 1
			elif event.key == pygame.K_v:
				# draw closed list
				if drawmode == 0:
					drawmode = 1
				else:
					drawmode = 0
			elif event.key == pygame.K_a:		# -------- A* Search --------
				choice = -1
				while int(choice) < 1 or int(choice) > 6:
					choice = raw_input ("Enter (1) for Manhattan distance, (2) for Euclidean distance, (3) for Octile distance, (4) for Chebyshev distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				MySearch = AStarSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,choice)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_u:		# -------- Uniform Cost Search --------
				MySearch = UniformCostSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,1)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_w:		# -------- Weighted A* Search --------
				choice = -1 #heuristic choice
				weight = 0 #weight of heuristic

				while (int(choice) < 1 or int(choice) > 6):
					choice = raw_input("Enter (1) for Manhattan distance, (2) for Euclidean Distance, (3) for Octile Distance, (4) for Chebyshev Distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				while float(weight) < 1:
					weight = raw_input("Enter the selected weight for Weighted A* - must be >= 1: ")
				
				MySearch = WeightedAStarSearch(weight)
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y, choice)
				elapsed_time = time.time() - start_time
				
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_q:		# -------- Sequential A* Search --------
				MySearch = AStarSearch()

				
		#print(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)
		drawScreen(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)

pygame.quit()

In [57]:
#tree

In [58]:
# for i in tree.items():
#         a = i[0]
#         b = i[1][0]
#         x,y = get_coord(str(b))
#         if back_track(x,y) in tree:
#             #print("has path",back_track(x,y))
#             print("\n")
#         else:
#             print("Not",i)

In [65]:
#print(value)
final_dict = {}
for i in tree.items():
    a = i[0]
    b = i[1]
    #print(a,b)
    if a and b:
        for item in b:
            final_dict[str(item)] = str(a)
print(final_dict)

{'62,68': '61,68', '62,69': '62,68', '63,69': '62,69', '64,69': '63,69', '64,68': '64,69', '64,67': '64,68', '63,67': '64,67', '62,67': '63,67', '62,66': '62,67', '61,66': '62,66', '60,66': '61,66', '59,66': '60,66', '58,66': '59,66', '58,65': '58,66', '57,65': '58,65', '57,64': '57,65', '56,64': '57,64', '55,64': '56,64', '55,63': '55,64', '56,63': '55,63', '57,63': '56,63', '58,63': '57,63', '58,64': '58,63', '59,64': '58,64', '60,64': '59,64', '60,63': '60,64', '59,63': '60,63', '59,62': '59,63', '60,62': '59,62', '61,62': '60,62', '61,63': '61,62', '62,63': '61,63', '62,62': '62,63', '63,62': '62,62', '64,62': '63,62', '64,61': '64,62', '64,60': '64,61', '65,60': '64,60', '66,60': '65,60', '66,59': '66,60', '65,59': '66,59', '65,58': '65,59', '66,58': '65,58', '67,58': '66,58', '68,58': '67,58', '69,58': '68,58', '69,57': '69,58', '68,57': '69,57', '68,56': '68,57', '69,56': '68,56', '70,56': '69,56', '70,55': '70,56', '71,55': '70,55', '72,55': '71,55', '72,54': '72,55', '72,53': 

In [59]:
get_validate_neighbour(0,0)

(1, 0)

In [60]:
get_neighbour(0,0,1)

(1, 0)

In [61]:
track_maze[0][1]

0

In [64]:
back_track(92,98)

Track this 92 98
add this to tree 99,94
tree has ['98,93']
Newlist ['98,93', '99,94']


backtrack returns 98,94


'98,94'